# [Travelbrag](http://travelbrag.ca)
Andrea & Brett's travelogue.

In [50]:
import pandas

colours = {"Andrea & Brett": "#00FA21", "Andrea": "#F2FA00", "Brett": "#005CFA"}

travelogue = pandas.read_csv("travelogue.csv")

In [51]:
travelogue.tail()

,year,purpose,travellers,city,country,latitude,longitude,arrived,departed,photos,comment
166,2015,PyCon,Andrea & Brett,Montréal,CAN,45.509000,-73.588000,2015-04-07,2015-04-15,https://goo.gl/photos/dxGD9qh69W4RR5iM6,Andrea didn't come until the 10th
167,2015,Visit Claire & Irving,Andrea & Brett,Montclair,USA,40.826000,-74.211000,2015-04-02,2015-04-07,https://goo.gl/photos/dxGD9qh69W4RR5iM6,NaN
168,2016,5-year wedding anniversary,Andrea & Brett,Tofino,CAN,49.152984,-125.906618,2016-02-12,2016-02-16,NaN,NaN
169,2016,5-year wedding anniversary,Andrea & Brett,Ucluelet,CAN,48.941600,-125.546345,2016-02-15,2016-02-15,NaN,NaN
170,2016,Python Day,Brett,Redmond,USA,47.674000,-122.122000,2016-04-24,2016-04-30,NaN,NaN


In [52]:
from bokeh import models

map_options = models.GMapOptions(lat=0, lng=0, map_type="terrain", zoom=1)

plot = models.GMapPlot(x_range=models.DataRange1d(), y_range=models.DataRange1d(), map_options=map_options,
                       title="Travelbrag")

plot.add_tools(models.PanTool(), models.WheelZoomTool())

In [53]:
import bokeh.io

bokeh.io.output_notebook(hide_banner=True)
bokeh.io.show(plot)

# Statistics

What cities have we visited?

In [75]:
import itertools


def grouper(iterable, n, fillvalue=None):
    """Collect data into fixed-length chunks or blocks.
    
    From https://docs.python.org/3/library/itertools.html#itertools-recipes.
    """
    args = [iter(iterable)] * n
    return itertools.zip_longest(*args, fillvalue=fillvalue)


locations = travelogue[['city', 'country', 'latitude', 'longitude']].drop_duplicates()
countries = locations['country'].drop_duplicates()
city_names = tuple(map(', '.join, ((row[0], row[1]) for row in locations.values.tolist())))
longest_name = max(len(name) for name in city_names)

print("We have visited {} cities in {} countries".format(len(locations), len(countries)))
print()
print("Countries:")
print('  ', ', '.join(countries.values.tolist()))
print()
print("Cities:")
for group in grouper(city_names, 4):
    print('  ', ' '.join(city.ljust(longest_name) for city in group if city is not None))

We have visited 98 cities in 12 countries

Countries:
   USA, CAN, GBR, JPN, BEL, CZE, ESP, FRA, CHE, CUB, ARG, DOM

Cities:
   Anaheim, USA          Yosemite Valley, USA  Sackville, CAN        Summerside, CAN      
   Yarmouth, CAN         Annapolis Royal, CAN  Amherst, CAN          Digby, CAN           
   Kentville, CAN        Moncton, CAN          Shediac, CAN          London, GBR          
   Brighton, GBR         York, GBR             Edinburgh, GBR        Oxford, GBR          
   Chicago, USA          Mammoth Lakes, USA    Washington, D.C., USA Las Vegas, USA       
   Montclair, USA        New York, USA         Atlantic City, USA    Calgary, CAN         
   Edmonton, CAN         Ely, GBR              Shoreham-by-Sea, GBR  Addison, USA         
   Pittsburgh, USA       Seattle, USA          Annaka, JPN           Hiroshima, JPN       
   Kyoto, JPN            Nara, JPN             Osaka, JPN            Takasaki, JPN        
   Tokyo, JPN            Brussels, BEL         Ghent, BE

# Data validation

In [55]:
import datetime
import re
import urllib.parse


# Year
if (travelogue['year'] < 1995).any() or (travelogue['year'] > datetime.date.today().year).any():
    raise ValueError('year not between 1995 and today')

# Travellers
if any(x not in colours for x in travelogue['travellers']):
    raise ValueError('unrecognized travellers:', unrecognized)
    
# City
if any(pandas.isnull(travelogue['city'])):
    raise ValueError('missing the city')

# Country
if any(len(x) != 3 or x.upper() != x for x in travelogue['country']):
    bad_countries = []
    raise ValueError('malformed country')
    
# Latitude
if any(lat < -90 or lat > 90 for lat in travelogue['latitude']):
    raise ValueError('malformed latitude')
    
# Longitude
if any(lng < -180 or lng > 180 for lng in travelogue['longitude']):
    raise ValueError('malformed longitude')
    
# Arrived
acceptable_date = re.compile(r'\d{4}(-\d{2}(-\d{2})?)?')
if any(not acceptable_date.match(x) for x in travelogue['arrived']):
    raise ValueError('malformed arrival date')
    
# Departed
if any(not acceptable_date.match(x) for x in travelogue['departed']):
    raise ValueError('malformed departure date')
    
# Photos
photo_urls = travelogue['photos'].dropna(how='any')
for url in photo_urls:
    parsed_url = urllib.parse.urlparse(url)
    if not parsed_url.scheme or not parsed_url.netloc or not parsed_url.path:
        raise ValueError('malformed photo URL:', url)
        
print('All travelogue data is valid!')

All travelogue data is valid!
